#### <center></center>
# <center><b>Отчёт по лабораторной работе №1</b></center>
Выполнили:

+ Горобец Иван M3236

+ Морозов Андрей M3236

+ Цвей Лев M3236

+ Илюхин Артем M3236

### <center>Постановка задачи</center>
Исходная задача представляет собой поиск точки минимума функции $f: O \subset \mathbb{R}^n → \mathbb{R}, \space f ∈ C^1(O)$. В нашей работе ограничимся случаями n = 1, 2. Общая стратегия отыскания точки экстремума будет следующей: задаём начальную точку $x_0$ далее строим последовательность точек $x_k$ следующим образом: $x_{k+1} = x_k - \nabla f(x_k) \cdot learning \_  rate$ - то есть каждый раз двигаемся по направлению антиградиента. Основной трудностью является оптимальный выбор $learning\_rate$ на каждом шаге.

### <center>Критерий остановки</center>
Также необходимо понять, в каких случаях нужно остановить вычисления. Так как мы ищем точку минимума $f(x)$, то для нас таковым будет условие $|∇ f(x)| < ε$, где $ε$ - достаточно маленькая числовая константа. Также поставим отсечку по количеству итераций на $10^6$ шагов.

### <center>Реализованные стратегии выбора шага</center>
+ `Constant`: $learning\_rate$ не изменяется и на каждом шаге равен изначально заданной константе.
+ `Polynomial decay`: $learning\_rate(k) = \displaystyle \frac 1 {\sqrt k}$
+ `Кусочно-постоянный`: $learning\_rate(k) = \displaystyle \frac 1 {2^m}, \space \frac 1 {2^m} \le k \le \frac 1 {2^{m-1}}$
+ `Экспоненциальный`: $learning\_rate(k) = \displaystyle \frac 1 {e^2} e^{-t k}$

## <center>Методы одномерного спуска</center>

+ правило Арамихо
+ правило Вольфе-Пауэлла
+ Дихотомия
+ Метод золотого сечения

_Каждый метод подробно описан в соответствующей ячейке_

In [ ]:
from copy import copy, deepcopy
from math import sqrt, sin, pow, log

import numpy as np
import matplotlib.pyplot as plt
import typing as tp
from collections.abc import Callable
import random
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
from scipy.optimize import minimize
import numdifftools as nd

-----
Блоки кода, отвечающие за необходимые импорты библиотек для корректной работы программы

In [ ]:
# Вынесенные общие типы для функций

# Определение типа функции одной переменной
Func1D = Callable[[float], float]
# Определение типа функции двух переменных
Func2D = Callable[[np.ndarray], float]
# Определение типа планировщика обучения
Scheduler = Callable[[Func2D, np.ndarray, int], float]
# Определение типа правила поиска
SearchRule = Callable[[Func1D, float, float], float]


In [ ]:
def grad_desc_plot_2d(
        func: Func2D,
        steps: np.ndarray,
        ax: plt.Axes,
        xlim: tuple[float, float],
        ylim: tuple[float, float],
        cmap: str = "viridis",
        title: str = ""
) -> None:
    """
    Функция отрисовки шагов градиентного спуска для функции 2х переменных.

    :param func: функция, которая минимизируется градиентным спуском
    :param steps: np.array[N x 2] — шаги алгоритма
    :param ax: холст для отрисовки графика
    :param xlim: tuple(float), 2 — диапазон по первой оси
    :param ylim: tuple(float), 2 — диапазон по второй оси
    :param cmap: str — название палитры
    :param title: str — заголовок графика
    """

    ax.set_title(title, fontsize=40, fontweight="bold")

    x_range = np.linspace(*xlim, 100)
    y_range = np.linspace(*ylim, 100)
    grid = np.meshgrid(x_range, y_range)
    X, Y = grid
    fvalues = func(
        np.dstack(grid).reshape(-1, 2)
    ).reshape((x_range.size, y_range.size))
    ax.pcolormesh(x_range, y_range, fvalues, cmap=cmap, alpha=0.8)
    CS = ax.contour(x_range, y_range, fvalues)
    ax.clabel(CS, CS.levels, inline=True)

    arrow_kwargs = dict(linestyle="--", color="black", alpha=0.8)
    for i, _ in enumerate(steps):
        if i + 1 < len(steps):
            ax.arrow(
                *steps[i],
                *(steps[i+1] - steps[i]),
                **arrow_kwargs
            )

    n = len(steps)
    color_list = [(i / n, 0, 0, 1 - i / n) for i in range(n)]
    ax.scatter(steps[:, 0], steps[:, 1], c=color_list, zorder=10)
    ax.scatter(steps[-1, 0], steps[-1, 1],
               color="red", label=f"estimate = {np.round(steps[-1], 2)}")

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_ylabel("$y$")
    ax.set_xlabel("$x$")
    ax.legend(fontsize=16)

Блок кода, содержащий функцию вычисления градиента

In [ ]:
def numeric_gradient(f: Func2D, x: np.ndarray) -> np.ndarray:
    """
    Функция для высчитывания значения градиента функции f в точке x с использованием центральных разностей.

    :param f: Анализируемая функция
    :type f: tp.Callable
    :param x: Точка
    :type x: np.array
    """
    eps = 2 ** (-52) # Machine epsilon for 64-bit doubles

    h_x = sqrt(eps) * x[0] if x[0] != 0 else sqrt(eps)
    h_y =  sqrt(eps) * x[1] if x[1] != 0 else sqrt(eps)

    approx_x = (f(x + np.array([h_x, 0])) -
              f(x - np.array([h_x, 0]))) / (2 * h_x)

    approx_y = (f(x + np.array([0, h_y])) -
              f(x - np.array([0, h_y]))) / (2 * h_y)

    return np.array([approx_x, approx_y])


Константная стратегия выбора шага. На каждой итерации алгоритма градиетного спуска всегда возвращает постоянное значение.

In [ ]:
def constant_scheduler(f: Func2D, x: np.ndarray, previous_lr: float = 0.01) -> float:
  """
  Возвращает постоянное значение learning rate (lr).

  :param f: Целевая функция
  :param x: Текущая точка
  :param previous_lr: Предыдущее значение lr
  :return: 0.01
  """
  return 0.01

**Экспоненциальная** стратегия выбора шага. Чем больше итераций алгоритма градиентного спуска было выполнено, тем сильнее затухает `learning_rate`.

In [ ]:
def exp_scheduler(start: float = 3e-2, t: float = 0.001) -> Scheduler:
  """
  Фабрика для создания экспоненциального планировщика learning rate.
  Возвращает функцию, которая вычисляет learning rate по формуле:
  lr(n) = start * exp(-t * n), где n - номер итерации.

  :param start: Начальное значение learning rate (по умолчанию 3e-2).
  :param t: Коэффициент скорости затухания (по умолчанию 0.001).
  :return: Функия, возвращающая значение learning rate для итерации n.
  """
  def wrapper_scheduler(f, x, n):
    return start * np.exp(-t * n)
  return wrapper_scheduler

**Полиномиальная** стратегия выбора шага. Данный метод позволяет сохранить высокую скорость спуска на начальном этапе и уменьшать `learning_rate` по мере приближения к ответу.

> Добавить блок с цитатой



In [ ]:
def polynom_scheduler(
        steps: int = 10000,
        init_lr: float = 0.05,
        end_lr: float = 0.0001,
        power: float = 0.5
) -> Scheduler:
    """
    Фабрика для создания полиномиального планировщика learning rate.
    Возвращает функцию, которая вычисляет learning rate.

    :param steps: Количество шагов между init_lr и end_lr (по умолчанию 10_000).
    :param init_lr: Стартовый learning rate (по умолчанию 0.01).
    :param end_lr: Минимальный learning rate (по умолчанию 0.0001).
    :param power: Степень полинома (по умолчанию 1).
    :return: Функия, возвращающая значение learning rate для итерации n.
    """

    def wrapper_scheduler(f, x, n):
      n = min(n, steps)
      return (init_lr - end_lr) * (1 - n / steps) ** (power) + end_lr

    return wrapper_scheduler

**Кусочно-постоянная** стратегия выбора шага

In [ ]:
def partial_const_scheduler(
    init_lr: float = 0.1,
    end_lr: float = 0.001,
    power: float = 2
) -> Scheduler:
    """
    Фабрика для создания кусочно-постоянного планировщика learning rate.
    Возвращает функцию, которая вычисляет learning rate.

    :param init_lr: Стартовый learning rate (по умолчанию 0.01).
    :param end_lr: Минимальный learning rate (по умолчанию 0.0001).
    :param power: Понижающий коэффициент (по умолчанию 2).
    :return: Функия, возвращающая значение learning rate для итерации n.
    """

    def wrapper_scheduler(f, x, n):
      n = int(log(n + 1, power))
      return max(init_lr * (1 / (power ** n)), end_lr)

    return wrapper_scheduler

# <center></center>
### **<center>Правило Арамихо</center>**

$$ f(x_k - \lambda \nabla f(x_k)) < f(x_k) - \sigma \lambda \space||\nabla f(x_k)||^2 \space $$
\- это условие Арамихо. Вычисление $learning\_rate$ не будет зависеть от номера текущего шага, а будет определяться следующим образом:

Изначально $\lambda = \lambda_0$, пока условие Арамихо не выполняется: $\lambda_{new} = \beta \cdot \lambda_{prev}$. Константы $\lambda_0, \space \beta, \space \sigma$, являются `hyper`параметрами, заданы изначально и не меняются в ходе работы.

In [ ]:
def armijo_rule(*, sigma: float = 0.3, beta: float = 0.8) -> Scheduler:
  """
  Фабрика для создания правила Армихо для выбора learning rate.

  :param sigma: Гипер параметр: условия достаточного убывания (по умолчанию 0.3).
  :param beta: Гипер параметр: Коэффициент уменьшения шага (по умолчанию 0.8).
  :return: Функция, принимающая (f, x, previous_lr) и возвращающая новый lr.
  """
  def wrapper(f, x, previous_lr = 1):
    lr = previous_lr
    p_k = -numeric_gradient(f, x)

    rule = lambda a: f(x + p_k * a) <= f(x) - \
                      sigma * lr * np.linalg.norm(-p_k) ** 2

    while not rule(lr):
      lr *= beta

    return lr

  return wrapper

# <center></center>
### **<center>Правило Вольфе-Пауэлла</center>**
Два условия:
 $$f(x_k - \lambda \nabla f(x_k)) \le f(x_k) - C_1 \lambda \space||\nabla f(x_k)||^2 \quad \quad (1)$$
 $$-\nabla f(x_k) \cdot \nabla f(x_k - \lambda \nabla f(x_k))  \le C_2 \space||\nabla f(x_k)||^2 \quad (2)$$
 как и в предыдущем правиле, пока не выполняется хотя бы одно из условий, меняем шаг: $\lambda_{new} = \beta \cdot \lambda_{prev}$.

 Константы $C_1, C_2, \beta \space -$ являются `hyper`параметрами, заданы изначально и не меняются в ходе работы.

In [ ]:
def wolfe_rule(*, C1: float = 1e-4, C2: float = 0.9, beta: float = 0.8) -> Scheduler:
  """
  Фабрика для создания правила Вольфе для выбора learning rate.

  :param C1: Гипер параметр: Параметр условия достаточного убывания (по умолчанию 1e-4).
  :param C2: Гипер параметр: Параметр условия кривизны (по умолчанию 0.9).
  :param beta: Гипер параметр: Коэффициент уменьшения шага (по умолчанию 0.8).
  :return: Функция, принимающая (f, x, previous_lr) и возвращающая новый lr.
  """
  def wrapper(f, x, previous_lr = 1):

    lr = previous_lr
    p_k = -numeric_gradient(f, x)

    rule_I = lambda a: f(x + p_k * a) <= f(x) - C1 * a * np.linalg.norm(p_k) ** 2
    rule_II = lambda a: np.dot(numeric_gradient(f, x + a * p_k), -p_k) \
                          >= -C2 * np.linalg.norm(p_k) ** 2

    while not rule_I(lr) or not rule_II(lr):
      lr *= beta
    return lr

  return wrapper


In [ ]:
# def goldstein_rule(*, dzeta_1 = 0.01, dzeta_2 = 0.85, beta = 0.5):
#   """
#   Фабрика для создания правила Гольдштейна для выбора learning rate.

#   :param dzeta_1: Гипер параметр: Верхний параметр условия (по умолчанию 0.01).
#   :param dzeta_2: Гипер параметр: Нижний параметр условия (по умолчанию 0.85).
#   :param beta: Гипер параметр: Коэффициент уменьшения шага (по умолчанию 0.5).
#   :return: Функция, принимающая (f, x, previous_lr) и возвращающая новый lr.
#   """
#   def wrapper(f, x, previous_lr = 1):

#     lr = previous_lr
#     p_k = -numeric_gradient(f, x)

#     rule = lambda a: dzeta_2 * (a * np.dot(p_k, -p_k)) <= \
#                      f(x + a * p_k) - f(x) <= \
#                      dzeta_1 * (a * np.dot(p_k, -p_k))

#     while not rule(lr):
#         lr *= beta
#     return lr

#   return wrapper


# <center></center>
### **<center>Метод дихотомии</center>**
Работа этого метода сильно опирается на унимодальность функции. \\
У нас есть отрезок, на котором мы ищем минимум. Для этого мы делим его каждый раз пополам, а потом сравниваем значение в средней точке с значениями в серединах получившихся маленьких отрезков. За счет унимодальности мы можем гарантированно сказать, в какой части изначального отрезка находится минимум, после чего сдвинуть его границы на новый отрезок \\
<big><strong>Плюсы данного алгоритма:</strong></big>
<ol>
<li>Простота реализации (всего 10 строк)</li>
<li>Он работает за меньшее количество шагов чем метод золотого сечения</li>
</ol>
<big><strong>Минусы:</strong></big>
<ol>
<li>Необходимость много раз считать значени функции в точке (если это долго, то могут быть проблемы с эфективностью алгоритма)</li>
</ol>

In [ ]:
def dihotomy_method_unimodal(f: Func1D, start: float, end: float, tol: float = 1e-6) -> float:
  """
  Поиск минимума унимодальной функции на отрезке при помощи метода дихотомии (половинным делением)

  :param f: унимодальная функция
  :param start: начало отрезка, на котором ищем минимум
  :param end: конец отрезка, на котором ищем минимум
  :param tol: точность (наибольшая разность между точкой в которой достигается минимум и возвращаемым значением)
  """

  while end - start > tol:
    mid = (end + start) / 2
    if f(mid) > f((start + mid) / 2):
      end = mid
    elif f(mid) > f((mid + end) / 2):
      start = mid
    else:
      start = (start + mid) / 2
      end = (end + mid) / 2

  return (start + end) / 2

# <center></center>
### **<center>Метод золотого сечения</center>**
Этот метод, как и предыдущий опирается на унимодальность функции, а также работает при помощи сужения отрезка в константное число раз на каждой итерации

<big><strong>Но есть два важных отличия:</strong><big>
<ol>
<li>В методе дихотомии длина отрезка каждый раз сокращалась в $2$ раза, а в этом методе в $\phi = \frac{1 + \sqrt{5}}{2} \approx 1.618$</li>
<li>Однако за счет этого делается меньше вычислений значения функции в точке!</li>
</ol>

За счет чего достигается второй пункт?

На кадом этапе, вместо того чтобы бить отрезок на 4 равные части и считать значения в каждой из 3 средних точке, мы будем бить его в отношении $1: \phi$, а также $1:(\phi-1)$. Таким образом мы получим две точки, сравнив значения в которых и значения на концах мы сможем понять на каком из отрезков находиться минимум. Однако! В отличие от прошлого метода, когда мы выберем этот отрезок, оба конца и одна из средних точек уже будет подсчитана, за счет того что длины отрезков относятся как золотое сечение. Это позволяет на каждой итерации делать ровно одно вычисление значения функции в точке, что может ускорить код, если это дорогостоящая операция

In [ ]:
def golden_ratio_method_unimodal(f: Func1D, start: float, end: float, tol: float = 1e-6) -> float:
    """
    Поиск минимума унимодальной функции на отрезке при помощи метода золотого сечения

    :param f: унимодальная функция
    :param start: начало отрезка, на котором ищем минимум
    :param end: конец отрезка, на котором ищем минимум
    :param tol: точность (наибольшая разность между точкой в которой достигается минимум и возвращаемым значением)
    """

    # константа золотого сечения
    phi: float = (1 + sqrt(5)) / 2
    # части на которые делится отрезок
    bigger_part: float = phi - 1
    smaller_part: float = 1 - bigger_part
    c = start + smaller_part * (end - start)
    d = start + bigger_part * (end - start)

    if (val1 := f(c)) > (val2 := f(d)):
        start = c
        # сохраняем результат для неиспользованной точки чтобы не пересчитывать его в будущем
        saved_dot: float = d
        saved_value: float = val2
        chose_left_on_last_choice: bool = True
    else:
        end = d
        saved_dot: float = c
        saved_value: float = val1
        chose_left_on_last_choice: bool = False

    while end - start > tol:
        new_dot = start + (end - start) * (bigger_part if chose_left_on_last_choice else smaller_part)
        if (new_value := f(new_dot)) < saved_value:
            if chose_left_on_last_choice:
                start = saved_dot
            else:
                end = saved_dot
            saved_value = new_value
            saved_dot = new_dot
        else:
            if chose_left_on_last_choice:
                end = new_dot
            else:
                start = new_dot
            chose_left_on_last_choice ^= True

    return (start + end) / 2

Блок кода, с реализацией алгоритма градиентого спуска.

In [ ]:
def grad_desc_with_rule(
        func: Func2D,
        start: np.ndarray,
        lr: Callable[..., float],
        max_iter: int = 10**6,
        eps: float = 1e-6
) -> tuple[np.ndarray, list[np.ndarray]]:
    """
    Finding function minimum using gradient descent.

    :param func: function to be analyzed
    :type func: Callable[np.array(x, y)]
    :param start: starting point
    :type start: np.array(x, y)
    :param lr: rule for learning rate calculation
    :type lr: Callable(func, np.array(x, y), float)
    :param max_iter: maximum amount of iterations
    :type max_iter: int
    :param eps: stop criteria
    :type eps: float

    :returns: minimum value and history of values
    """

    x = start
    history = [x]

    for i in range(max_iter):
        if "scheduler" in lr.__name__:
            learning_rate = lr(func, x, i)
        else:
            learning_rate = lr(func, x)
        gradient = numeric_gradient(func, x)
        x_new = x - learning_rate * gradient
        if np.linalg.norm(gradient) < eps:
            break
        x = x_new
        history.append(x)

    return x, history

In [ ]:
def grad_desc_on_1dsearch(
        func: Func2D,
        start: np.ndarray,
        search_rule: SearchRule,
        max_iter: int = 10**8,
        eps: float = 1e-6,
        lr_min: float = 0,
        lr_max: float = 1
) -> tuple[np.ndarray, list[np.ndarray]]:
    """
    Finding function minimum using gradient descent.
    Learning rate is calculated used a specified 1d_search algorithm

    :param func: function to be analyzed
    :type func: Callable[np.array(x, y)]
    :param start: starting point
    :type start: np.array(x, y)
    :param search_rule: rule for learning rate calculation
    :type lr: Callable(func, np.array(x, y), float)
    :param max_iter: maximum amount of iterations
    :type max_iter: int
    :param eps: stop criteria
    :type eps: float

    :returns: minimum value and history of values
    """


    x = start
    history = [x]

    for i in range(max_iter):
        gradient = numeric_gradient(func, x)
        func_to_minimize = lambda lr: func(x - gradient * lr)
        learning_rate = search_rule(func_to_minimize, lr_min, lr_max)
        x_new = x - learning_rate * gradient
        if np.linalg.norm(gradient) < eps:
            break
        x = x_new
        history.append(x)

    return x, history

Блок кода, генерирующий на основе заданной функции функцию с шумом.

In [ ]:
def func_with_noise(func: Func2D, noise_coeff: float = 2) -> Func2D:
  def wrapper_func(x):
    return func(x) + np.random.random(2) * noise_coeff

Основной блок кода, показывающий принцип работы алгоритмов на различных функциях и визуализирующий сам процесс.

In [ ]:
test_functions = {
    "Quadratic_1": lambda x: x[0] * x[0] + x[1] * x[1] - 6 * x[0] - 8 * x[1] + 10,
    "Quadratic_2": lambda x: 2 * x[0] * x[0] + 3 * x[1] * x[1] - 12 * x[0] - 18 * x[1] + 20,
    "Quadratic_3": lambda x: 4 * x[0] * x[0] + 5 * x[1] * x[1] + 2 * x[0] * x[1] - 16 * x[0] - 20 * x[1] + 15,

    "Multimodal_trigonom": lambda x: sin(0.5 * x[0]) * sin(0.5 * x[0]) + sin(0.5 * x[1]) * sin(0.5 * x[1]),
    "Hard function_1": lambda x: - 1 / (x[0] * x[0] + x[1] * x[1] + 1) - 1 / pow(
        (x[0] - 2) * (x[0] - 2) + (x[1] - 2) * (x[1] - 2) + 1, 2),

    "Himmelblau": lambda x: pow(x[0] * x[0] + x[1] - 11, 2) + pow(x[0] + x[1] * x[1] - 7, 2),
}

rules = {
    "Constant_scheduler": constant_scheduler,
    "Exp_scheduler": exp_scheduler(),
    "Polynom_scheduler": polynom_scheduler(),
    "Partial_const_scheduler": partial_const_scheduler(),
    "Armijo_rule": armijo_rule(),
    "Wolfe_rule": wolfe_rule()
}

search_1d = {
    "Dihotomy": dihotomy_method_unimodal,
    "Golden_ratio": golden_ratio_method_unimodal
}

In [ ]:
# for func_name, func in test_functions.items():
#     point = np.random.randn(2)

#     fig = plt.figure(figsize=(50, 40), constrained_layout=True)

#     gs = GridSpec(3, 1, height_ratios=[0.1, 0.8, 1.4], figure=fig)

#     title_ax = fig.add_subplot(gs[0])
#     title_ax.axis('off')
#     title_ax.set_title(func_name, fontsize=55, fontweight="bold")

#     gs_3d = GridSpecFromSubplotSpec(1, 3, subplot_spec=gs[1], wspace=0)
#     ax_3d = fig.add_subplot(gs_3d[0, 1], projection='3d')

#     x = np.linspace(-5, 5, 100)
#     y = np.linspace(-5, 5, 100)
#     X, Y = np.meshgrid(x, y)
#     Z = np.array([func([xi, yi]) for xi, yi in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

#     surf = ax_3d.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha=0.8, linewidth=0, antialiased=True)

#     ax_3d.set_title("3D Visualization", fontsize=40)
#     ax_3d.set_xlabel('X', fontsize=12, labelpad=10)
#     ax_3d.set_ylabel('Y', fontsize=12, labelpad=10)
#     ax_3d.set_zlabel('Z', fontsize=12, labelpad=10)
#     ax_3d.view_init(elev=30, azim=45)

#     cbar = fig.colorbar(surf, ax=ax_3d, shrink=0.6, aspect=20, pad=0.05)
#     cbar.ax.tick_params(labelsize=10)

#     inner_gs = GridSpecFromSubplotSpec(3, 3, subplot_spec=gs[2], wspace=0.01, hspace=0.01)

#     ax_2d = []
#     for i in range(3):
#         for j in range(3):
#             ax_2d.append(fig.add_subplot(inner_gs[i, j]))
#             ax_2d[-1].set_aspect('equal', adjustable='box')

#     i = 0

#     for rule_name, rule in rules.items():
#         try:
#           res, search_log = grad_desc_with_rule(func, point, rule)
#           search_points = np.vstack(search_log)
#         except Exception as e:
#           print(rule_name)
#           print(e)
#           print("-------------------------------------------------")
#         grad_desc_plot_2d(
#             np.vectorize(func, signature="(n)->()"),
#             search_points,
#             ax=ax_2d[i],
#             xlim=(-5, 5),
#             ylim=(-5, 5),
#             title=rule_name
#         )

#         i += 1

#     for search_name, search in search_1d.items():
#         res, search_log = grad_desc_on_1dsearch(func, point, search)
#         search_points = np.vstack(search_log)

#         grad_desc_plot_2d(
#             np.vectorize(func, signature="(n)->()"),
#             search_points,
#             ax=ax_2d[i],
#             xlim=(-5, 5),
#             ylim=(-5, 5),
#             title=search_name
#         )
#         i += 1

#     for ax in ax_2d:
#         ax.grid(True, linestyle='--', alpha=0.7)
#         ax.tick_params(axis='both', which='major', labelsize=10)

#     plt.show()
#     print("_________________________________________________")


Из библиотеки `SciPy` были использованы методы `BFGS` (алгоритм Бройдена–Флетчера–Гольдфарба–Шанно) и `CG` (метод сопряжённых градиентов)



In [ ]:
def compute_direction(f, x):
    grad = numeric_gradient(f, x)
    try:
        return np.dot(-np.linalg.inv(nd.Hessian(f)(x)), grad)
    except np.linalg.LinAlgError:
        return -grad

In [ ]:
# f = list(test_functions.items())[0]
# print(nd.Hessian(f[1])(np.array([1.0, 2.0])))

def newton_cg(
        func: Func2D,
        start: np.ndarray,
        sch: Callable[..., float],
        max_iter: int = 10**6,
        eps: float = 1e-6
) -> tuple[np.ndarray, list[np.ndarray]]:
    """
    Finding function minimum using gradient descent.

    :param func: function to be analyzed
    :type func: Callable[np.array(x, y)]
    :param start: starting point
    :type start: np.array(x, y)
    :param lr: rule for learning rate calculation
    :type lr: Callable(func, np.array(x, y), float)
    :param max_iter: maximum amount of iterations
    :type max_iter: int
    :param eps: stop criteria
    :type eps: float

    :returns: minimum value and history of values
    """

    x = start
    history = [x]
    for i in range(max_iter):
        if "scheduler" in sch.__name__:
            learning_rate = sch(func, x, i) * 100
        else:
            learning_rate = sch(func, x)

        direction = compute_direction(func, x)
        x_new = x + learning_rate * direction
        if np.linalg.norm(numeric_gradient(func, x)) < eps:
            break
        # print(np.linalg.norm(numeric_gradient(func, x)))
        x = x_new
        history.append(x)

    return x, history

# from functools import partial
# initial_point = (-0.0031, 0.206)
# min_ = (3, 2)
# for name, f in [list(test_functions.items())[5]]:
#   for method in ["BFGS", "CG"]:
#     result = minimize(f, initial_point, method='BFGS', jac=partial(numeric_gradient, f))
#     print(f"Результат {method} на функции {name}:")
#     print("Минимум:", result.x)
#     print(np.linalg.norm(result.x - min_))
#     print("Количество итераций:", result.nit)
#     print("Количество вычислений функции:", result.nfev)
#     print("Количество вычислений градиента:", result.njev)
#     print()
#   print("_________________")
#   print()

In [ ]:
f = list(test_functions.items())[1]
print(f)
# print(nd.Hessian(f[1])(np.array([1.0, 2.0])))
ans = newton_cg(f[1], np.array([0, 0]), constant_scheduler)
print(ans[0], len(ans[1]))

('Quadratic_2', <function <lambda> at 0x795f7ea25080>)
[3. 3.] 2


----
# Тестовая секция

### Тест функции `Quadratic_1` : $x ^ 2 + y ^ 2 - 6x - 8 y + 10$
Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(-0.96, -0.167)` | 801 | 4.97e-7 | `(3, 4)`
`Exp scheduler`| `(-0.96, -0.167)` | 306 | 4.73e-7 | `(3, 4)`
`Polynom scheduler`| `(-0.96, -0.167)` | 306 | 4.73e-7 | `(3, 4)`
`Partial const scheduler`| `(-0.96, -0.167)` | 7280 | 5.1e-3 | `(3, 4)`
`Armijo`| `(-0.96, -0.167)` | 14 | 3.34e-7 | `(3, 4)`
`Wolfe`| `(-0.96, -0.167)` | 33 | 4.49e-7 | `(3, 4)`
`Dihotomy`| `(-0.96, -0.167)` | 2 | 6.51e-8 | `(3, 4)`
`Golden ratio`| `(-0.96, -0.167)` | 3 | 4.27e-8 | `(3, 4)`
`BFGS `| `(-0.96, -0.167)` | 3 | 1.09e-7 | `(3, 4)`
`CG `| `(-0.96, -0.167)` | 3 | 1.09e-7 | `(3, 4)`

<br>

-------

<br>


### Тест функции `Quadratic_2`: $2 x ^ 2 + 3 y ^ 2 - 12 x -18y + 20$


Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(1.16, -0.32)` | 408 | 2.42e-7 | `(3, 3)`
`Exp scheduler`| `(1.16, -0.32)` | 141 | 2.38e-7  | `(3, 3)`
`Polynom scheduler`| `(1.16, -0.32)` | 71 | 1.37e-5 | `(3, 3)`
`Partial const scheduler`| `(1.16, -0.32)` | 3226 | 8.49e-5 | `(3, 3)`
`Armijo`| `(1.16, -0.32)` | 14 | 8.76e-8 | `(3, 3)`
`Wolfe`| `(1.16, -0.32)`  | 33 | 9.07e-8 | `(3, 3)`
`Dihotomy`| `(1.16, -0.32)` | 12 | 6.16e-8 | `(3, 4)`
`Golden ratio`| `(1.16, -0.32)` | 12 | 6.22e-8 | `(3, 3)`
`BFGS `| `(1.16, -0.32)` | 5 | 8.14e-09 | `(3, 3)`
`CG `| `(1.16, -0.32)` |5 | 8.14e-09 | `(3, 3)`


<br>

-------

<br>

### Тест функции `Quadratic_3`: $4 x ^ 2 + 5 y ^ 2 + 2xy -16 x - 20 y + 15$
Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(0.53, -0.72)` | 211 | 1.55e-7  | `(30/19, 32/19)`
`Exp scheduler`|`(0.53, -0.72)` | 69| 1.32e-7  | `(30/19, 32/19)`
`Polynom scheduler`| `(0.53, -0.72)` | 37 | 1.28e-7 | `(30/19, 32/19)`
`Partial const scheduler`| `(0.53, -0.72)` | 1476 | 4.76e-7 | `(30/19, 32/19)`
`Armijo`| `(0.53, -0.72)`| 13  | 5.06e-8 | `(30/19, 32/19)`
`Wolfe`|  `(0.53, -0.72)`|137 | 7.58e-8 | `(30/19, 32/19)`
`Dihotomy`| `(0.53, -0.72)`| 7 | 5.08e-8 | `(30/19, 32/19)`
`Golden ratio`| `(0.53, -0.72)`| 7 | 2.88e-8 | `(30/19, 32/19)`
`BFGS `| `(0.53, -0.72)` | 5 | 1.17e-08 | `(30/19, 32/19)`
`CG `| `(0.53, -0.72)` | 5 | 1.17e-08 | `(30/19, 32/19)`

<br>

-------

<br>


### Тест функции `Multimodal_trigonom`: $\sin(0.5 \cdot x) ^ 2 + \sin(0.5 \cdot y) ^ 2$
Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(0.22, -0.52)` | 2513 | 1.99e-6 | `(0, 0)`
`Exp scheduler`| `(0.22, -0.52)` | 1805 | 1.99e-6 | `(0, 0)`
`Polynom scheduler`| `(0.22, -0.52)` | 540 | 1.22e-6 | `(0, 0)`
`Partial const scheduler`| `(0.22, -0.52)` | 26307 | 1.22e-6 | `(0, 0)`
`Armijo`| `(0.22, -0.52)` | 22  | 1.17e-6 | `(0, 0)`
`Wolfe`| `(0.22, -0.52)` | 21 | 1.57e-6 | `(0, 0)`
`Dihotomy`| `(0.22, -0.52)` | 23 |1.17e-6  | `(0, 0)`
`Golden ratio`| `(0.22, -0.52)` | 20 | 1.57e-6 | `(0, 0)`
`BFGS `| `(0.22, -0.52)` | 4 | 1.96e-5 | `(0, 0)`
`CG `| `(0.22, -0.52)` |4 | 1.96e-5 | `(0, 0)`


<br>

-------

<br>


### Тест функции `Hard_function_1`: $-\frac 1 {x ^ 2 + y ^ 2 + 1} - \frac 1 {((x-2) ^ 2 + (y - 2)  ^ 2  + 1)^2}$
Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(-1.38, -0.28)` |881 | 0.007 | `(0, 0)`
`Exp scheduler`|  `(-1.38, -0.28)`| 347 | 0.006 | `(0, 0)`
`Polynom scheduler`|  `(-1.38, -0.28)`| 228 | 0.005 | `(0, 0)`
`Partial const scheduler`|  `(-1.38, -0.28)`| 10914 | 0.005 | `(0, 0)`
`Armijo`| `(-1.38, -0.28)` | 12 | 0.0078 | `(0, 0)`
`Wolfe`|  `(-1.38, -0.28)` | 23 | 0.0075 | `(0, 0)`
`Dihotomy`| `(-1.38, -0.28)` | 9 | 0.005 | `(0, 0)`
`Golden ratio`| `(-1.38, -0.28)` | 8 | 0.0056 | `(0, 0)`
`BFGS `| `(-1.38, -0.28)` | 4 | 0.007 | `(0, 0)`
`CG `| `(-1.38, -0.28)` |4 | 0.007 | `(0, 0)`


<br>

-------

<br>


### Тест функции `Himmelblau`: $(x ^ 2 + y - 11) ^ 2 + (x + y ^ 2 - 7) ^ 2$
Метод | Старт | Количество итераций | Расстояние до минимума | Ближайшая точка минимума
:-|:-:|:-:|:-:|:-:
`Constant scheduler`| `(-0.0031, 0.206)` | 64 | 3.88e-8| `(3, 2)`
`Exp scheduler`| `(-0.0031, 0.206)` |337 | 1.08e-8 | `(3, 2)`
`Polynom scheduler`| `(-0.0031, 0.206)` | - | - | `(3, 2)`
`Partial const scheduler`| `(-0.0031, 0.206)` | - | - | `(3, 2)`
`Armijo`| `(-0.0031, 0.206)` | 30 | 1.43e-8 | `(3, 2)`
`Wolfe`| `(2.039, -0.004)` | 117 | 1.159e-8 | `(3, 2)`
`Dihotomy`| `(-0.0031, 0.206)` | 17 | 2.01e-8 | `(3, 2)`
`Golden ratio`| `(-0.0031, 0.206)` | 16 | 2.69e-8 | `(3, 2)`
`BFGS `| `(-0.0031, 0.206)` | 10 | 7.08e-10 | `(3, 2)`
`CG `| `(-0.0031, 0.206)` | 12 | 7.08e-10 | `(3, 2)`



# <center></center>
### **<center>Заключение</center>**

Как и ожидалось, хуже всего себя показал `Constant sheduler` (особенно на сложных функциях, порядка 2500 итераций в худшем случае). Немногим лучше работал выбор шага с экпоненциальным затуханием - 1800 итераций для функции `Multimodal_trigonom`. `Polynomial sheduler` и `Partial const sheduler`не нашли экстремум функции `Himmelblau`. Правило Вольфе-Пауэла в среднем работало за 30 итераций, но на функциях `Himmelblau` и `Quadratic_3` число итераций превышало 100. Стоит отметить, что на функциях с большим количеством точек перегиба правила Арамихо и Вольфе-Пауэлла могли работать как за 15-20 итераций (в лучшем случае), так и за >100 итераций. Самыми стабильными оказались метод золотого сечения и метод дихотомии.